# Pipeline completo: Entrenamiento ResNet-18, XAI y Evaluación Quantitativa

Este notebook unifica los tres pasos principales del TFM:

1. **Entrenamiento de ResNet-18 sobre MedMNIST** (equivalente a `train.py`).
2. **Generación de explicaciones XAI** con Grad-CAM / Grad-CAM++ / Integrated Gradients / Saliency (equivalente a `xai_explanations.py`).
3. **Evaluación cuantitativa de la explicabilidad con Quantus** (equivalente a `quantus_evaluation.py`).

La idea es poder ejecutar de principio a fin todo el pipeline desde un único notebook, con código comentado y secciones claras.


## 0. Configuración inicial

En esta sección:
- Comprobamos la versión de Python y PyTorch.
- Configuramos el dispositivo (CPU / GPU).
- Definimos rutas básicas y semilla de reproducibilidad.



In [1]:
import os
import json
import random
from pathlib import Path

import numpy as np
import torch

# Ruta del proyecto
PROJECT_DIR = Path('/home/TFM_Laura_Monne')
RESULTS_DIR = PROJECT_DIR / 'results'
DATA_DIR    = PROJECT_DIR / 'data'

# Dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Dispositivo:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# Semilla global para reproducibilidad
def set_global_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_global_seed(42)


Dispositivo: cpu


## 1. Entrenamiento / carga del modelo ResNet-18

El **Objetivo** es **cargar y visualizar** los artefactos generados por los scripts ejecutados desde la **terminal** (`prepare_data.py`, `train.py`, `quick_test.py`).
En la carpeta results/ tenemos los archivos `training_results.json`, `training_history.png`, `confusion_matrix.png`, `preds_test.npz`, `best_model.pth`.

In [2]:
# Notebook para cargas y visualizar los resultados del entrenamiento de ResNet-18

from pathlib import Path
import sys
import subprocess
import platform

# Ruta del proyecto
PROJECT_DIR = Path('/home/TFM_Laura_Monne')
RESULTS_DIR = PROJECT_DIR / 'results'
DATA_DIR    = PROJECT_DIR / 'data'

print('Proyecto :', PROJECT_DIR)
print('Resultados:', RESULTS_DIR)
print('Datos     :', DATA_DIR)

assert PROJECT_DIR.exists(), f"No existe {PROJECT_DIR}. Clona el repo desde TERMINAL antes de usar el notebook."
assert RESULTS_DIR.exists(), f"No existe {RESULTS_DIR}. Entrena desde TERMINAL (python train.py) antes de usar el notebook."

def ensure(pkg_import_name, pip_name=None):
    """Importa o instala con pip si falta."""
    try:
        __import__(pkg_import_name)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pip_name or pkg_import_name])

# Dependencias mínimas para este notebook
ensure("numpy")
ensure("pandas")
ensure("sklearn", "scikit-learn")

Proyecto : \home\TFM_Laura_Monne
Resultados: \home\TFM_Laura_Monne\results
Datos     : \home\TFM_Laura_Monne\data


AssertionError: No existe \home\TFM_Laura_Monne. Clona el repo desde TERMINAL antes de usar el notebook.



Aquí tenemos dos opciones:

- **Opción A (rápida, recomendada)**: cargar el mejor modelo ya entrenado desde `results/best_model.pth` (si ya ejecutaste `train.py` o el notebook de entrenamiento).
- **Opción B**: entrenar desde cero (puede tardar bastante; el código está resumido y se basa en `train.py`).

Primero intentaremos cargar el modelo entrenado. Si no existe el checkpoint, puedes ejecutar el entrenamiento reducido desde este notebook.


In [ ]:
from prepare_data import load_datasets
from train import Trainer, create_data_loaders
from resnet18 import create_model

RESULTS_DIR = PROJECT_ROOT / "results"
RESULTS_DIR.mkdir(exist_ok=True)
BEST_MODEL_PATH = RESULTS_DIR / "best_model.pth"

print("Ruta de checkpoint esperada:", BEST_MODEL_PATH)



In [ ]:
# Opción A: cargar modelo ya entrenado

if BEST_MODEL_PATH.exists():
    print("Encontrado checkpoint entrenado, cargando...")
    checkpoint = torch.load(BEST_MODEL_PATH, map_location=device)
    model = create_model(num_classes=15)
    model.load_state_dict(checkpoint["model_state_dict"])
    model.to(device)
    model.eval()
    print("Modelo cargado correctamente.")
else:
    print("❌ No se ha encontrado best_model.pth. Ejecuta la celda de entrenamiento (Opción B).")
    model = None


### 1.C Visualización de curvas de entrenamiento

Igual que en el *Notebook 1. Entrenamiento ResNet-18*, aquí mostramos las curvas de:

- Pérdida y precisión por época (`training_history.png`, generada por `Trainer.plot_history()`).
- (Opcional) Curvas guardadas por `make_report_assets.py` (`fig_training_curves.png`).

Esta sección se ejecuta justo después del entrenamiento/carga del modelo para que puedas inspeccionar fácilmente cómo ha ido el entrenamiento.


In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

results_dir = Path("results")

fig_paths = [
    results_dir / "training_history.png",
    results_dir / "fig_training_curves.png",
]

for p in fig_paths:
    if p.exists():
        print(f"Mostrando figura de entrenamiento: {p}")
        img = Image.open(p)
        plt.figure(figsize=(8, 4))
        plt.imshow(img)
        plt.axis("off")
        plt.show()
    else:
        print(f"(Info) No se encontró la figura: {p}. Ejecuta el entrenamiento completo o 'make_report_assets.py' si la necesitas.")


### 1.B Entrenamiento reducido (opcional)

Si **no** tienes un modelo entrenado, puedes ejecutar esta celda para entrenar una versión reducida (menos épocas) solo para comprobar que el pipeline funciona.

> Nota: para los resultados finales del TFM se recomienda entrenar con `train.py` completo (120 épocas, early stopping, etc.). Aquí usamos muy pocas épocas para ir rápido.


In [ ]:
# Entrenamiento reducido (ejecutar solo si model es None)

if model is None:
    print("Entrenando modelo reducido (pocas épocas, solo para prueba)...")

    # Config similar a train.py pero reducido
    config = {
        "batch_size": 64,
        "epochs": 5,  # reducido
        "learning_rate": 1e-3,
        "weight_decay": 1e-4,
        "early_stopping_patience": 3,
        "num_workers": 4,
        "use_class_weights": True,
        "grad_clip_norm": 1.0,
        "num_classes": 15,
    }

    # Cargar datasets y dataloaders
    datasets = load_datasets("./data", target_size=224)
    train_loader, val_loader, test_loader, class_weights_vec = create_data_loaders(
        datasets,
        batch_size=config["batch_size"],
        num_workers=config["num_workers"],
        num_classes=config["num_classes"],
    )

    model = create_model(num_classes=config["num_classes"]).to(device)
    trainer = Trainer(
        model,
        train_loader,
        val_loader,
        test_loader,
        device,
        config,
        class_weights=class_weights_vec if config["use_class_weights"] else None,
    )

    history = trainer.train()
    trainer.save_model(str(BEST_MODEL_PATH))
    print("Checkpoint reducido guardado en:", BEST_MODEL_PATH)
else:
    print("Ya hay un modelo cargado; se omite el entrenamiento reducido.")


## 2. Generación de explicaciones XAI (Grad-CAM, Grad-CAM++, IG, Saliency)

En esta sección reutilizamos la clase `XAIExplainer` de `xai_explanations.py` para:

- Seleccionar unas pocas imágenes del conjunto de **test**.
- Obtener la predicción del modelo.
- Generar explicaciones con:
  - Grad-CAM
  - Grad-CAM++
  - Integrated Gradients
  - Saliency Maps
- Mostrar las imágenes originales junto con sus mapas de explicabilidad.

Esto corresponde funcionalmente al script `xai_explanations.py`, pero aquí lo hacemos de forma interactiva.


In [ ]:
from data_utils import create_data_loaders_fixed
from xai_explanations import XAIExplainer

# Asegurarnos de tener un modelo entrenado/cargado
aassert_msg = "El modelo es None. Asegúrate de haber cargado o entrenado en la sección 1."
assert model is not None, aassert_msg

# DataLoaders con el collate robusto a canales
datasets = load_datasets("./data", target_size=224)
_, _, test_loader = create_data_loaders_fixed(
    datasets=datasets,
    batch_size=1,
    num_workers=0,
    seed=42,
)

explainer = XAIExplainer(model, device, num_classes=15)
print("XAIExplainer inicializado.")


In [ ]:
import matplotlib.pyplot as plt

# Número de ejemplos que queremos visualizar
NUM_EXAMPLES = 3

examples = []
for idx, (data, target) in enumerate(test_loader):
    if idx >= NUM_EXAMPLES:
        break
    examples.append((data.to(device), target.to(device)))

print(f"Tomados {len(examples)} ejemplos del conjunto de test.")

for i, (x, y_true) in enumerate(examples):
    with torch.no_grad():
        logits = model(x)
        pred_class = int(logits.argmax(dim=1).item())
        true_class = int(y_true.item())
    print(f"Ejemplo {i}: true={true_class}, pred={pred_class}")

    # Generar todas las explicaciones (reutiliza la interfaz ya existente)
    res = explainer.generate_all_explanations(
        input_tensor=x,
        pred_class=pred_class,
        image_idx=i,
    )

print("Mapas XAI guardados en carpeta 'outputs/' (gradcam, gradcampp, integrated_gradients, saliency).")


## 4. Visualización de curvas de entrenamiento

En esta sección mostramos, si existen, las figuras generadas durante el entrenamiento:

- `results/training_history.png` (generada por `Trainer.plot_history()` en `train.py`).
- `results/fig_training_curves.png` (generada por `make_report_assets.py`).

La celda de abajo **no falla** si los ficheros aún no existen: simplemente indica cuáles faltan.


In [ ]:
from pathlib import Path
import matplotlib.pyplot as plt
from PIL import Image

fig_paths = [
    Path("results") / "training_history.png",
    Path("results") / "fig_training_curves.png",
]

for p in fig_paths:
    if p.exists():
        print(f"Mostrando figura: {p}")
        img = Image.open(p)
        plt.figure(figsize=(8, 4))
        plt.imshow(img)
        plt.axis("off")
        plt.show()
    else:
        print(f"(Info) No se encontró la figura: {p}")


## 3. Evaluación cuantitativa de la explicabilidad con Quantus

Finalmente, evaluamos de forma **cuantitativa** las explicaciones generadas, usando **Quantus**.

En lugar de llamar a `quantus_evaluation.py` desde consola, reutilizamos su lógica simplificada directamente aquí:

- Tomamos un pequeño lote del conjunto de test (por ejemplo, 30 imágenes).
- Generamos las atribuciones para cada método XAI.
- Calculamos las métricas de Quantus:
  - FaithfulnessCorrelation
  - AvgSensitivity
  - Complexity / Entropy
  - MPRT / ModelParameterRandomisation
  - RegionPerturbation
- Guardamos los resultados en un JSON para poder analizarlos o introducirlos en la memoria.


In [ ]:
from quantus_evaluation import (
    collect_samples,
    evaluate_methods,
    save_results,
)

# Número de muestras para la evaluación cuantitativa (puedes subirlo si quieres más robustez)
NUM_SAMPLES_QUANTUS = 30

# Volvemos a crear un loader de test (batch_size=1 para recolectar muestras fácilmente)
_, _, test_loader_q = create_data_loaders_fixed(
    datasets=datasets,
    batch_size=1,
    num_workers=0,
    seed=42,
)

x_batch, y_batch = collect_samples(test_loader_q, NUM_SAMPLES_QUANTUS, device)
print("Forma de x_batch:", x_batch.shape)
print("Forma de y_batch:", y_batch.shape)

methods_to_eval = ["gradcam", "gradcampp", "integrated_gradients", "saliency"]

results = evaluate_methods(
    model=model,
    explainer=explainer,
    x_batch=x_batch,
    y_batch=y_batch,
    methods=methods_to_eval,
    device=device,
)

OUTPUT_QUANTUS = PROJECT_ROOT / "outputs" / "quantus_metrics_notebook.json"
save_results(results, str(OUTPUT_QUANTUS))

results
